# 1.2.2: Bikeshare (Implementing the State)

<br>



---



*Modeling and Simulation in Python*


Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---





Most models attempt to describe some system that changes over time.  To simulate this process of change, we need two things:

1) A set of variables that describe the state of the system at any given moment (i.e. the *state variables*)

2) A rule for how these variables change with each *time step*.  This is called a *change function*.

### Storing the *state* in a Series

The state is defined by a set of values which will change over the course of the simulation.  In order to store this state, we'll use an `object` called a `Series`, which is part of a `Library` of code called Pandas.  A couple definitions:

- An `object` is a catch-all term for a "thing" in python: a collection of data of any sort.

- A `Series` is an object that holds a set of variable values in a table-like form: it's a particular kind of `object`.  We'll see an example of this in action in just a moment.

- A `Library` is a set of pre-written code that we can access in our own programs.  You can "check out" this pre-written code by "importing" it into your code.

We'll start by importing the Pandas library (which is a data analysis library), and creating a `Series` called `bikeshare`.  This will be our *state object*: it will hold information about the state of our system.

In [ ]:
import pandas as pd

state = dict(augie=10,moline=2)
bikeshare = pd.Series(state,name='Number of Bikes')
bikeshare


Notice the format here:

- The first line imports the library, and gives it the shorthand `pd`.  Importing the library makes all of its objects and methods available to the program, and the shorthand version slims down the code a bit.

- The second line creates a foundational Python object called a `dictionary`, which is a set of label-value pairs (think of it like a word and a definition--thus the name dictionary!).

Together, the two labels ('Augie' and 'Moline') represent our *state variables*.  The *initial state* indicates that there are 10 bikes at Augie and 2 at Moline.

- The third line creates the `Series` and assigns it to a variable name `bikeshare`.  `bikeshare` now refers to the Series, in the same way that `g` refers to `9.8` when we write `g = 9.8`. A `Series` is a lot like a dictionary, but it is much easier to manipulate and is a more powerful way to store data.  The `name` is a *keyword argument* that describes the meaning of our values: '2' and '10' represent the number of bikes at the two locations.  

- The final line sends the contents of `bikeshare` to be output, which is why the details of the Series appear when we run the cell.

We can read the variables inside a `Series` using the *dot operator*, like this:

In [ ]:
bikeshare.augie

And this:

In [ ]:
bikeshare.moline

Or, to display the state variables and their values, you can just enter the name of the object:

In [ ]:
bikeshare

We can represent our `Series` in a way that looks more like a table by transforming the `Series` into another Pandas object called a `DataFrame` (we'll get to those later: we're just using it as a display tool here):

In [ ]:
pd.DataFrame(bikeshare)

You don't have to use this format,  but there a couple reasons to use it.  One, the results look better.  Two, it provides a better visual sense of the nature of a `Series`.  But most importantly, Colab turns a `DataFrame` into an interactive table.  

<br>

After you've run the cell, click the little calculator icon.  Now click 'Number of Bikes': see how it rearranges the table?  Click filter, and play with that.  If you click the chart icon (three vertical lines), it will even suggest different plots you could make and provide the code to do that. Obviously, we don't really need this tool here, but it might come in handy later!

### The change function

Ok, we now have an initial state.  But any useful model is going to involve change.

<br>

In a model, the process of change must be defined by a rule, or a set of rules, about how this change occurs.  We will call these rules a *change function*.  Much of this class will be spent exploring different types of change functions.  These rules might stay the same the whole time, or they might change, gradually or suddenly.  The rules might create random *stochastic* changes, or established *deterministic* changes.

<br>

The change occurs through *time steps*.  Each time step represents a certain amount of time, and each step represents a change in the state of the system.  In the previous notebook, we defined our time step as being 15 minutes.

At its most basic, we enact the change function manually: that is, we can update the state by assigning new values to the variables.
For example, if a student moves a bike from Augie to Moline, we can figure out the new values and assign them:

In [ ]:
# First simple change function
bikeshare.augie = 9
bikeshare.moline = 3
bikeshare

augie     9
moline    3
Name: Number of Bikes, dtype: int64

That's not particularly efficient, though.  One step better, we can avoid doing the math ourselves by using *update operators*, `-=` and `+=`, to subtract 1 from
`augie` and add 1 to `moline`:

In [ ]:
# Second simple change function
bikeshare.augie -= 1
bikeshare.moline += 1
bikeshare

augie     8
moline    4
Name: Number of Bikes, dtype: int64

Try running the last cell again.  What happens to the state of the system?  Now run the previous code cell (with the title "first simple change function") again.  Finally, run the last cell again.

<br>

Each time you run the cell, it performs the action described by the code, even if you had already run that cell before.  This can be useful, but it can also cause trouble if you are not paying close attention!

### Using a Python Function

The advantage of using a computer to create a model (i.e. of creating *simulations*) is that we don't have to manually enter the change for each time step.  We want to define the rule (that is, the change function) and let the computer do the grunt work.

<br>

Oftentimes, we will want to code the change function in a Python `function`.  This allows us to write a few lines of code, test them to confirm they do what we intend, and then use that function for every time step.

<br>

For example, these lines move a bike from Augie to Moline:

In [ ]:
# Not inside a function
bikeshare.augie -= 1
bikeshare.moline += 1

Rather than repeat them every time a bike moves, we can define a new
function:

In [ ]:
# Inside a function
def bike_to_moline():
    bikeshare.augie -= 1
    bikeshare.moline += 1

`def` is a special word in Python that indicates we are defining a new
function. The name of the function is `bike_to_moline`. Notice some of the details:

<br>

* The empty
parentheses indicate that this function requires no additional
information when it runs.
* The colon indicates the beginning of an
indented *code block*.
* The next two lines are the *body* of the function. They *have* to be indented; by convention, the indentation is four spaces.  Always follow this convention!
* Notice the name of the function: always choose a name that describes what a function does, as this makes reading and editing the code much easier.

<br>

When you define a function, it has no immediate effect, because all you have done is to create a set of steps (i.e. the function).

<br>

Open the variable window `{x}` and run the "Inside a function" cell repeatedly.  Confirm that the values of `bikeshare` do not change.  Now run the cell "Not inside a function" a couple times.  Check to see that the values of `bikeshare` change.

The "instructions" in the function are not followed until you *call* the function. Here's how to call
this function.  Notice there is no colon or `def`:

In [ ]:
bike_to_moline()
bikeshare

augie      2
moline    10
Name: Number of Bikes, dtype: int64

When you call the function, it runs the steps in the body of the function, which
update the variables of the `bikeshare` object.

When you call a function, you have to include the parentheses. If you
leave them out, you get this:

In [ ]:
bike_to_moline

<function __main__.bike_to_moline()>

This result indicates that `bike_to_moline` is a function. You don't have to know what `__main__` means, but if you see something like this, it probably means that you named a function but didn't actually call it.
So don't forget the parentheses.

<br>

The chief benefit of defining functions is that you avoid repeating chunks of
code, which makes programs smaller and much easier to read, edit, and debug.




### Using `print` to monitor a simulation

As you write more complicated programs, it is easy to lose track of what
is going on. One of the most useful tools for debugging is the *print statement*, which displays text in cell output.

<br>

Normally when Colab runs the code in a cell, it displays the value of
the last line of code. For example, if you run:

In [ ]:
bikeshare.augie
bikeshare.moline

10

Jupyter runs both lines, but it only displays the value of the
second. If you want to display more than one value, you can use
print statements:

In [ ]:
print(bikeshare.augie)
print(bikeshare.moline)

2
10


When you call the `print` function, you can print a single data object or you can provide a sequence separated by commas, like this:

In [ ]:
print("There are", bikeshare.augie,"bikes at Augustana, and",
      bikeshare.moline, "bikes in Moline.")

There are 2 bikes at Augustana, and 10 bikes in Moline.


Python looks up the values of the variables and displays them. Notice that when you put letters or numbers inside quotation marks, the letters/ numbers are not treated as variables or numerical values.  These are called `strings`.

<br>

Print statements are useful for debugging functions. For example, we can
add a print statement to `move_bike`, like this:

In [ ]:
def bike_to_moline():
    print('Before Moving a bike to Moline', bikeshare.moline)
    bikeshare.augie += 1
    bikeshare.moline -= 1
    print('After moving a bike to Moline', bikeshare.moline)

Each time we call this version of the function, it displays a message,
which can help us keep track of what the program is doing.
 If I were to call this function, and the value of `bikeshare.moline` did not change, or became smaller, I would know that something was off with the code in between the two print statements.


In [ ]:
bike_to_moline()
bikeshare

Before Moving a bike to Moline 8
After moving a bike to Moline 7


augie     5
moline    7
Name: Number of Bikes, dtype: int64

Oops!  Looks like our code is taking a bike away from moline instead of adding one.  Go above and fix the code to be correct!

<br>



---

## Summary and Exercises

This chapter introduces the tools we need to keep track of the state of a system and to change that state.

In the next chapter, we'll use these tools to create our first simulation, which will show the change in the system over the course of a day.



### Exercise 1

✅ ✅  What happens if you spell the name of a state variable wrong?  Edit the following cell, change the spelling of `moline`, and run it.

The error message uses the word *attribute*, which is another name for what we are calling a state variable.

In [ ]:
bikeshare = pd.Series(dict(augie=10,moline=2),name="Number of Bikes")

bikeshare.moline

### Exercise 2

✅ ✅  Make a state object, but this time add a third state variable in addition to `augie` and `moline`.  Call the third variable `rock_island`, with initial value 0, and display the state of the system.

In [ ]:
# Solution goes here
